# Laboratoire 2 de CEG 4536


In [40]:
!nvidia-smi

Tue Nov  5 02:29:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [41]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


# **Partie 1** :  Profiling

In [54]:
%%writefile Tache4_Optimisation1.cu
#include <stdio.h>
#include <cuda.h>

__global__ void optimizewithKernel(int *input, int *output, int size) {
    extern __shared__ int sharedData[];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Charge les éléments dans la mémoire partagée
    if (idx < size) {
        sharedData[tid] = input[idx];
    } else {
        sharedData[tid] = 0;
    }
    __syncthreads();

    // Réduction parallèle
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    // Le premier thread de chaque bloc stocke le résultat
    if (tid == 0) {
        output[blockIdx.x] = sharedData[0];
    }
}

int main() {
    const int size = 1024;
    int *h_input, *h_output, *d_in, *d_out;
    h_input = (int*)malloc(size * sizeof(int));
    h_output = (int*)malloc(sizeof(int));

    // Initialisation des données
    for (int i = 0; i < size; i++) {
        h_input[i] = 1;
    }

    // Allocation de la mémoire sur le GPU
    cudaMalloc(&d_in, size * sizeof(int));
    cudaMalloc(&d_out, sizeof(int));

    // Copie des données de l'hôte vers le GPU
    cudaMemcpy(d_in, h_input, size * sizeof(int), cudaMemcpyHostToDevice);

    // Lancer le kernel
    optimizewithKernel<<<4, 256, 256 * sizeof(int)>>>(d_in, d_out, size);

    // Copie du résultat du GPU vers l'hôte
    cudaMemcpy(h_output, d_out, sizeof(int), cudaMemcpyDeviceToHost);

    // Affichage du résultat
    printf("Sum: %d\n", *h_output);

    // Libération de la mémoire
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_input);
    free(h_output);

    return 0;
}


Overwriting Tache4_Optimisation1.cu


In [55]:
!nvcc Tache4_Optimisation1.cu -o Tache4_Optimisation1

In [56]:
!nvprof ./Tache4_Optimisation1

==27018== NVPROF is profiling process 27018, command: ./Tache4_Optimisation1
Sum: 256
==27018== Profiling application: ./Tache4_Optimisation1
==27018== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.71%  4.9600us         1  4.9600us  4.9600us  4.9600us  optimizewithKernel(int*, int*, int)
                   25.00%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   16.29%  1.3760us         1  1.3760us  1.3760us  1.3760us  [CUDA memcpy HtoD]
      API calls:   68.05%  129.19ms         2  64.593ms  7.2080us  129.18ms  cudaMalloc
                   31.68%  60.134ms         1  60.134ms  60.134ms  60.134ms  cudaLaunchKernel
                    0.11%  209.41us         2  104.70us  13.908us  195.50us  cudaFree
                    0.11%  203.23us       114  1.7820us     249ns  77.310us  cuDeviceGetAttribute
                    0.04%  75.226us         2  37.613us  36.110us  39.116us  

# **Partie 2** : Optimisation pour maximiser l'occupation des warps et minimiser les latences

In [57]:
%%writefile Tache4_maximisation.cu
#include <stdio.h>
#include <cuda.h>

__global__ void optimizedKernelLatency(int *input, int *output, int size) {
    extern __shared__ int sharedData[];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < size) {
        sharedData[tid] = input[idx];
    } else {
        sharedData[tid] = 0;
    }
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        output[blockIdx.x] = sharedData[0];
    }
}

int main() {
    const int size = 1024;
    int *h_input, *h_output, *d_input, *d_output;

    h_input = (int*)malloc(size * sizeof(int));
    h_output = (int*)malloc(sizeof(int));

    for (int i = 0; i < size; i++) {
        h_input[i] = 1;
    }

    cudaMalloc(&d_input, size * sizeof(int));
    cudaMalloc(&d_output, sizeof(int));

    cudaMemcpy(d_input, h_input, size * sizeof(int), cudaMemcpyHostToDevice);

    int threadsPerBlock = 128; // Ajustement de la taille pour maximiser l'occupation
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    optimizedKernelLatency<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, size);

    cudaMemcpy(h_output, d_output, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Sum: %d\n", *h_output);

    free(h_input);
    free(h_output);
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Writing Tache4_maximisation.cu


In [58]:
!nvcc Tache4_maximisation.cu -o Tache4_maximisation

In [59]:
!nvprof ./Tache4_maximisation

==28516== NVPROF is profiling process 28516, command: ./Tache4_maximisation
Sum: 128
==28516== Profiling application: ./Tache4_maximisation
==28516== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.75%  4.5760us         1  4.5760us  4.5760us  4.5760us  optimizedKernelLatency(int*, int*, int)
                   26.19%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   17.06%  1.3760us         1  1.3760us  1.3760us  1.3760us  [CUDA memcpy HtoD]
      API calls:   71.49%  99.406ms         2  49.703ms  4.5400us  99.401ms  cudaMalloc
                   28.21%  39.227ms         1  39.227ms  39.227ms  39.227ms  cudaLaunchKernel
                    0.12%  171.51us         2  85.756us  13.904us  157.61us  cudaFree
                    0.10%  143.41us       114  1.2570us     134ns  58.026us  cuDeviceGetAttribute
                    0.06%  77.027us         2  38.513us  31.715us  45.312us

# Nous pouvons souligner a partir de cette tache 🇰

**Analyse des Résultats et Documentation** :  L'utilisation de nvprof a révélé des différences notables entre la version initiale et la version optimisée du kernel. Le temps total d'exécution sur le GPU est passé de 4.9600 µs dans la version de base à 4.5760 µs dans la version optimisée, montrant une gestion plus efficace des opérations. De plus, le temps pour cudaMalloc a été réduit de 129.19 ms à 99.406 ms, soulignant une meilleure allocation de la mémoire. L'ajustement de la taille des blocs et l'optimisation de l'occupation des warps ont contribué à diminuer la latence sans affecter les transferts de données (cudaMemcpy).

***Rapport d'Amélioration L'optimisation a permis plusieurs avancées ***: un temps de calcul réduit et une meilleure répartition des charges ont amélioré l'exécution du kernel. L'alignement des tailles de blocs sur la taille du warp a maximisé l'occupation et réduit les cycles inactifs, augmentant l'efficacité globale. L'utilisation de la mémoire partagée et la synchronisation des threads ont renforcé la scalabilité, permettant au programme de gérer de plus grandes quantités de données sans dégrader les performances.


*Pour conclure* on peut souligner que : *texte en italique*

**Pour le Profiling de la 1ere optimisation** : L'utilisation du profiling a permis d'identifier les goulets d'étranglement et d'évaluer les performances pour optimiser le programme.

**le profiling a partir d'une Optimisation basée sur les profil**s : Les techniques appliquées ont maximisé l'occupation des warps et réduit la latence, améliorant l'efficacité et la scalabilité pour des traitements intensifs sur GPU.